In [1]:
%env METAFLOW_PROFILE=oleg2

env: METAFLOW_PROFILE=oleg2


# Run the server

```
serve run server:batch_preds
```

In [4]:
import requests
from base import TabularBatchPrediction as TBP
import pandas as pd
import json

In [5]:
hello_world_output = requests.post("http://localhost:8000").json()
assert hello_world_output['response'] == "Hello World!", "Are you sure you're running the server?"

In [6]:
def get_data() -> pd.DataFrame:
    tbp = TBP()
    _, valid_dataset, test_dataset = tbp.load_dataset()
    df = test_dataset.to_pandas()
    df["id"] = df.index
    return df, valid_dataset.to_pandas()["target"].values


def prepare_post_body(batch: pd.DataFrame) -> dict:
    id_to_batch = {}
    for record in batch:
        _id = record.pop("id")
        id_to_batch[_id] = record
    return id_to_batch


def query_predict_endpoint(batch_size=5, df=None):
    if df is None:
        df = get_data()
    batch = df.sample(batch_size).to_dict("records")
    id_to_batch_features = prepare_post_body(batch)
    output = requests.post(
        "http://localhost:8000/predict/", data=json.dumps(id_to_batch_features)
    ).json()
    return output

In [7]:
df, true_targets = get_data()

No dataset name or loader provided. Using default breast_cancer.csv dataset.


2023-09-14 17:46:22,925	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
2023-09-14 17:46:24,521	INFO read_api.py:374 -- To satisfy the requested parallelism of 20, each read task output will be split into 20 smaller blocks.


[dataset]: Run `pip install tqdm` to enable progress reporting.


2023-09-14 17:46:25,970	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)]
2023-09-14 17:46:25,971	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-14 17:46:25,971	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


In [8]:
msg = "Prediction proba for sample input {} is {}. True target is {}."
output = query_predict_endpoint(batch_size=5, df=df)

In [9]:
for id, proba in output.items():
    print(msg.format(id, proba, true_targets[int(id)]))

Prediction proba for sample input 100 is 0.11638931185007095. True target is 0.
Prediction proba for sample input 55 is 0.8022734522819519. True target is 1.
Prediction proba for sample input 155 is 0.8228343725204468. True target is 1.
Prediction proba for sample input 75 is 0.8228343725204468. True target is 1.
Prediction proba for sample input 126 is 0.8797837495803833. True target is 1.


# What is running on the `/predict` endpoint on the server?

In [10]:
from ray.train.xgboost import XGBoostPredictor
import ray
from ray.train.batch_predictor import BatchPredictor
from metaflow import Run, Flow


def select_from_checkpoint_registry(flow_name="Train"):
    flow = Flow(flow_name)
    run = flow.latest_successful_run
    result = run.data.result
    return result.checkpoint


# duplicated in `query_predict_endpoint` above
batch_size = 2
if df is None:
    df = get_data()
batch = df.sample(batch_size).to_dict("records")
id_to_batch_features = prepare_post_body(batch)

features = ray.data.from_items(list(id_to_batch_features.values()))
checkpoint = select_from_checkpoint_registry()
predictor = BatchPredictor.from_checkpoint(checkpoint, XGBoostPredictor)
preds = predictor.predict(features).to_pandas()["predictions"].values
preds_payload = dict(zip(id_to_batch_features.keys(), preds))

2023-09-14 17:47:14,732	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(StandardScaler._transform_pandas)->MapBatches(ScoringWrapper)]
2023-09-14 17:47:14,734	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-14 17:47:14,735	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`
2023-09-14 17:47:14,766	INFO actor_pool_map_operator.py:117 -- MapBatches(StandardScaler._transform_pandas)->MapBatches(ScoringWrapper): Waiting for 1 pool actors to start...
2023-09-14 17:47:15,747	WARNING actor_pool_map_operator.py:267 -- To ensure full parallelization across an actor pool of size 1, the specified batch size should be at most 2. You

In [11]:
preds_payload

{54: 0.89597195, 155: 0.89597195}